In [1]:
import numpy as np
import scipy as sp
import math
import Universality_check as Uc
import Gate_helper

### States

In [2]:
zero  = np.array([[1], [0], [0], [0], [0], [0]],dtype=complex)
one   = np.array([[0], [1], [0], [0], [0], [0]],dtype=complex)
two   = np.array([[0], [0], [1], [0], [0], [0]],dtype=complex)
three = np.array([[0], [0], [0], [1], [0], [0]],dtype=complex)
four  = np.array([[0], [0], [0], [0], [1], [0]],dtype=complex)
five  = np.array([[0], [0], [0], [0], [0], [1]],dtype=complex)

### Permutations

In [3]:
Id = zero@zero.T + one@one.T + two@two.T + \
     three@three.T + four@four.T + five@five.T
## Permutations for two species of fermions
SN_f = - zero@zero.T - one@one.T + five@two.T \
     +four@three.T+three@four.T+ two@five.T

SS_f =  -zero@zero.T - one@one.T + five@three.T \
     +four@two.T+two@four.T+ three@five.T

SE_f =  -two@two.T - three@three.T + five@one.T \
     +four@zero.T+zero@four.T+ one@five.T

SW_f = - two@two.T - three@three.T + five@zero.T \
     +four@one.T+one@four.T+ zero@five.T

D1_f = zero@three.T + three@zero.T +one@two.T + two@one.T \
     -four@four.T - five@five.T

D2_f = zero@two.T + two@zero.T +one@three.T + three@one.T \
     -four@four.T - five@five.T

## Permutations for two species of Bosons
SN_b =  zero@zero.T + one@one.T + five@two.T \
     +four@three.T+three@four.T+ two@five.T

SS_b =  zero@zero.T + one@one.T + five@three.T \
     +four@two.T+two@four.T+ three@five.T

SE_b =  two@two.T + three@three.T + five@one.T \
     +four@zero.T+zero@four.T+ one@five.T

SW_b =  two@two.T + three@three.T + five@zero.T \
     +four@one.T+one@four.T+ zero@five.T

D1_b =  zero@three.T + three@zero.T +one@two.T + two@one.T \
     +four@four.T + five@five.T

D2_b =  zero@two.T + two@zero.T +one@three.T + three@one.T \
     +four@four.T + five@five.T

## Permutations for blue (bottom) boson and green (top) of fermion
SN_m =  -zero@zero.T + one@one.T + five@two.T \
     +four@three.T+three@four.T+ two@five.T

SS_m =  zero@zero.T - one@one.T + five@three.T \
     +four@two.T+two@four.T+ three@five.T

SE_m =  -two@two.T + three@three.T + five@one.T \
     +four@zero.T+zero@four.T+ one@five.T

SW_m =  two@two.T - three@three.T + five@zero.T \
     +four@one.T+one@four.T+ zero@five.T

D1_m =  zero@three.T + three@zero.T +one@two.T + two@one.T \
     -four@four.T + five@five.T

D2_m =  zero@two.T + two@zero.T +one@three.T + three@one.T \
     +four@four.T - five@five.T

### Gate Set (S)

In [104]:
## Fermionic puzzle
Perms_f = [SN_f,SS_f,SE_f,SW_f]
SQRT_SWAP_f = [(Id + 1j*P)/np.sqrt(2) for P in Perms_f]
Quarter_Swap_f = [np.cos(np.pi/8)*Id + np.sin(np.pi/8)*1j*P for P in Perms_f]
S_f = SQRT_SWAP_f

## Bosonic puzzle
Perms_b = [SN_b,SS_b,SE_b,SW_b]
SQRT_SWAP_b = [(Id + 1j*P)/np.sqrt(2) for P in Perms_b]
Quarter_Swap_b = [np.cos(np.pi/8)*Id + np.sin(np.pi/8)*1j*P for P in Perms_b]
S_b = SQRT_SWAP_b

## Mixed puzzle
Perms_m = [SN_m,SS_m,SE_m,SW_m]# we add diagonal swaps here to get universality.
SQRT_SWAP_m = [(Id + 1j*P)/np.sqrt(2) for P in Perms_m]
Quarter_Swap_m = [np.cos(np.pi/8)*Id + np.sin(np.pi/8)*1j*P for P in Perms_m]
S_m = SQRT_SWAP_m

In [105]:
equal = 1/np.sqrt(6)*(zero+one+two+three+four+five)
np.abs(SQRT_SWAP_f[2]@SQRT_SWAP_f[0]@equal)
#1/2*(-SE_f@SN_f+1j*SN_f+1j*SE_f+Id)@equal
#np.abs(SQRT_SWAP_b[2]@SQRT_SWAP_b[2]@SQRT_SWAP_b[2]@SQRT_SWAP_b[3]@SQRT_SWAP_b[2]@equal)

array([[0.        ],
       [0.        ],
       [0.40824829],
       [0.40824829],
       [0.57735027],
       [0.57735027]])

## Step 1 of the algorithm,

For our gate set to be universal we need the center of our group to be the trivial center which is just scalar multiples of the identity. If there is a non identity element in the center than that element will commute with everything in our gate set and can thus not be synthesised by a sequence of gates. If the center is nontrivial than the groups may still be infinite but they cannot be universal.

In [106]:
print("Center is trivial for fermionic puzzle:",Uc.check_Center(S_f))
print("Center is trivial for bosonic puzzle:",Uc.check_Center(S_b))
print("Center is trivial for mixed puzzle:",Uc.check_Center(S_m))

Center is trivial for fermionic puzzle: False
Center is trivial for bosonic puzzle: False
Center is trivial for mixed puzzle: False


### Step 2 of the algorithm

This code determines the span of our gate set is infinite or finite. It does this by attempting to find an element that can be reached from our gate set that is in a ball of radius 1 that is not in the center of the group. Not if this check pases, but the previous check fails that the span of $S$ is infinite, but not all of $SU(6)$, meaning it is not universal.  

In [123]:
# using a much smaller value for N than is required to be thourough since i want this to actually run 
N_SU6 = 100  #36398100 # upper bound for N

print("Fermionic puzzle is infinite:",not Uc.check_Finite(S_f,N_SU6,10,Verbose=False)[0])
print("Bosonic puzzle is infinite:",not Uc.check_Finite(S_b,N_SU6,10,Verbose=False)[0])
print("Mixed puzzle is infinite:",not Uc.check_Finite(S_m,N_SU6,10,Verbose=False)[0])

Fermionic puzzle is infinite: True
Bosonic puzzle is infinite: True
Mixed puzzle is infinite: True


## Or all together we can check for universality


In [108]:
print("Fermionic puzzle is univeral:",Uc.check_universal(S_f))
print("Bosonic puzzle is univeral:",Uc.check_universal(S_b))
print("Mixed puzzle is univeral:",Uc.check_universal(S_m))

Fermionic puzzle is univeral: False
Bosonic puzzle is univeral: False
Mixed puzzle is univeral: False


## Sanity check on known universal gate sets for quibts

In [130]:
qubitzero  = np.array([[1], [0]],dtype=complex)
qubitone   = np.array([[0], [1]],dtype=complex)

#Clifford Group
S_gate = np.matrix(np.diag([1,1j]))
H = 1/math.sqrt(2)*np.matrix([[1,1],[1,-1]])
S_c = [S_gate,H]
## expected size of qubit clifford group is 24

#Pauli Group
X = np.matrix([[0,1],[1,0]])
Y = np.matrix([[0,-1j],[1j,0]])
Z = np.matrix(np.diag([1,-1]))
S_p = [X,Y,Z]

#Clifford + T 
T_gate = np.matrix(np.diag([1,np.exp(1j*np.pi/4)]))
S_t = [S_gate,H,T_gate]

In [135]:
print('Clifford Group is size:',Uc.check_Finite(S_c,100,10,Verbose=False)[1])
print('Pauli Group is size: ', Uc.check_Finite(S_p,1000,10,Verbose=False)[1])
print('Clifford + T Group is infinite:',not Uc.check_Finite(S_t,100,10,Verbose=False)[0])
print('--------------')
print('Clifford Group is univeral:',Uc.check_universal(S_c))
print('Pauli Group is univeral:',Uc.check_universal(S_p))
print('Clifford + T Group is universal:',Uc.check_universal(S_t))

Clifford Group is size: 24
Pauli Group is size:  4
Clifford + T Group is infinite: True
--------------
Clifford Group is univeral: False
Pauli Group is univeral: False
Clifford + T Group is universal: True


## Same sanity check but for qutrits

In [136]:
S_gate = Gate_helper.S_gate(3)
H = Gate_helper.H_gate(3)
S_c = [S_gate,H]

In [138]:
print('Clifford Group is size:',Uc.check_Finite(S_c,100,15,Verbose=False)[1])
print('Clifford Group is univeral:',Uc.check_universal(S_c))

Clifford Group is size: 216
Clifford Group is univeral: False


# Puzzle variants that achieve universality

## 1) Half Swaps with and an S gate

In [109]:
## add S_gate to fermionic gate set,
S_f1 = [G for G in SQRT_SWAP_f]
S_f1.append(Gate_helper.S_gate(6))

S_b1 = [G for G in SQRT_SWAP_b]
S_b1.append(Gate_helper.S_gate(6))

S_m1 = [G for G in SQRT_SWAP_m]
S_m1.append(Gate_helper.S_gate(6))

In [110]:
print("With S gate the fermionic puzzle is universal:",Uc.check_universal(S_f1))
print("With S gate the bosonic puzzle is universal:",Uc.check_universal(S_b1))
print("With S gate the mixed puzzle is universal:",Uc.check_universal(S_m1))

With S gate the fermionic puzzle is universal: True
With S gate the bosonic puzzle is universal: True
With S gate the mixed puzzle is universal: True


## 2) Add in Diagonal swaps so the puzzle allows all to all swaps

In [111]:
Perms_f2 = [SN_f,SS_f,SE_f,SN_f,D1_f,D2_f]
SQRT_SWAP_f2 = [(Id + 1j*P)/np.sqrt(2) for P in Perms_f2]
S_f2 = SQRT_SWAP_f2

Perms_b2 = [SN_b,SS_b,SE_b,SN_b,D1_b,D2_b]
SQRT_SWAP_b2 = [(Id + 1j*P)/np.sqrt(2) for P in Perms_b2]
S_b2 = SQRT_SWAP_b2

Perms_m2 = [SN_m,SS_m,SE_m,SN_m,D1_m,D2_m]
SQRT_SWAP_m2 = [(Id + 1j*P)/np.sqrt(2) for P in Perms_m2]
S_m2 = SQRT_SWAP_m2

In [112]:
print("With diagonal swaps the fermionic puzzle is universal:",Uc.check_universal(S_f2))
print("With diagonal swaps the bosonic puzzle is universal:",Uc.check_universal(S_b2))
print("With diagonal swaps the mixed puzzle is universal:",Uc.check_universal(S_m2))

With diagonal swaps the fermionic puzzle is universal: False
With diagonal swaps the bosonic puzzle is universal: False
With diagonal swaps the mixed puzzle is universal: True


### Quarter Swaps and Eight Swaps

In [114]:
S_f3 = Quarter_Swap_f
S_b3 = Quarter_Swap_b
S_m3 = Quarter_Swap_m

Eighth_Swap_f = [np.cos(np.pi/16)*Id + np.sin(np.pi/16)*1j*P for P in Perms_f]
Eighth_Swap_b = [np.cos(np.pi/16)*Id + np.sin(np.pi/16)*1j*P for P in Perms_b]
Eighth_Swap_m = [np.cos(np.pi/16)*Id + np.sin(np.pi/16)*1j*P for P in Perms_m]

S_f4 = Eighth_Swap_f
S_b4 = Eighth_Swap_b
S_m4 = Eighth_Swap_m

In [115]:
print("With quarter swaps the fermionic puzzle is universal:",Uc.check_universal(S_f3))
print("With quarter swaps the bosonic puzzle is universal:",Uc.check_universal(S_b3))
print("With quarter swaps the mixed puzzle is universal:",Uc.check_universal(S_m3))

print("With eighth swaps the fermionic puzzle is universal:",Uc.check_universal(S_f4))
print("With eighth swaps the bosonic puzzle is universal:",Uc.check_universal(S_b4))
print("With eighth swaps the mixed puzzle is universal:",Uc.check_universal(S_m4))

With quarter swaps the fermionic puzzle is universal: False
With quarter swaps the bosonic puzzle is universal: False
With quarter swaps the mixed puzzle is universal: False
With eighth swaps the fermionic puzzle is universal: False
With eighth swaps the bosonic puzzle is universal: False
With eighth swaps the mixed puzzle is universal: False
